# Fourier Ring Correlation

Developing code for Fourier Ring Correlation for a project inspired by a recent paper on 3D resolution evaluation in CryoEM.

*Tan, Yong Zi, Philip R Baldwin, Joseph H Davis, James R Williamson, Clinton S Potter, Bridget Carragher, and Dmitry Lyumkis. “Addressing Preferred Specimen Orientation in Single-Particle Cryo-EM through Tilting.” Nature Methods 14, no. 8 (July 3, 2017): 793–96. doi:10.1038/nmeth.4347.*

In the paper they introduce a method that is able to deal with anisotropic resolution. For this purpose they modified the traditional FSC, by introducing an inclinatino angle term. 

## Indexing

The authors provided their implementation of the algorithm no GitHub. However, that was quite horrible and I decided to rewrite the whole thing, based on my earlier FRC implementation. The first step isto figure out the indexing, as in order to add the angle terms $\theta$ and $\phi$ for inclination and azimuth I need to create an iterator that can work with spherical coordinates. For this purpose the first step is to create such a coordinate system.

In [45]:
import numpy as np
import supertomo.processing.ops_array as ops_array

# Create a cylindrical coordinate grid, based on the shape of an image.
dims = (5,5,5)
axes = (np.arange(-np.floor(i / 2.0), np.ceil(i / 2.0)) for i in dims)
z, x, y = np.meshgrid(*axes)

#Create OP vector array
r = np.sqrt(x**2+y**2+z**2)
        
# Create inclination and azimuth angle arrays
phi = np.arctan2(y, x)
theta = np.arccos(ops_array.safe_divide(z, r))

print ("Successfully created the coordinate matrixes for spherical indexing.")

pyculib:1: ImportWarning: Numba version too old; expecting 0.33.0


Successfully created the coordinate matrixes for spherical indexing.


/Users/sami/Programming/GIT/supertomo2/supertomo/processing/ops_array.py:190: RuntimeWarning: invalid value encountered in divide
  result = numerator / denominator


/Users/sami/anaconda/envs/supertomo2/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in divide
  app.launch_new_instance()


array([[[ 2.18627604,  2.30052398,  2.35619449,  2.30052398,  2.18627604],
        [ 1.91063324,  1.99133066,  2.03444394,  1.99133066,  1.91063324],
        [ 1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633],
        [ 1.23095942,  1.15026199,  1.10714872,  1.15026199,  1.23095942],
        [ 0.95531662,  0.84106867,  0.78539816,  0.84106867,  0.95531662]],

       [[ 2.30052398,  2.52611294,  2.67794504,  2.52611294,  2.30052398],
        [ 1.99133066,  2.18627604,  2.35619449,  2.18627604,  1.99133066],
        [ 1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633],
        [ 1.15026199,  0.95531662,  0.78539816,  0.95531662,  1.15026199],
        [ 0.84106867,  0.61547971,  0.46364761,  0.61547971,  0.84106867]],

       [[ 2.35619449,  2.67794504,  3.14159265,  2.67794504,  2.35619449],
        [ 2.03444394,  2.35619449,  3.14159265,  2.35619449,  2.03444394],
        [ 1.57079633,  1.57079633,  1.57079633,  1.57079633,  1.57079633],
        [ 1.10714872,